In [0]:

df_silver = spark.sql("select * from parquet.`abfss://silver@datastorager.dfs.core.windows.net/car_sales`")

In [0]:
df_silver.display()

In [0]:
df_dealer = spark.sql("select * from cars_catalog.gold_schema.dim_dealer")
df_branch = spark.sql("select * from cars_catalog.gold_schema.dim_branch")
df_model = spark.sql("select * from cars_catalog.gold_schema.dim_model")
df_date = spark.sql("select * from cars_catalog.gold_schema.dim_date")



bringing keys and joining

In [0]:
df_fact= df_silver.join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, "left")\
    .join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, "left")\
    .join(df_model, df_silver.Model_ID == df_model.Model_ID, "left")\
    .join(df_date, df_silver.Date_ID == df_date.Date_ID, "left")\
    .select(df_silver["Revenue"],df_silver["Units_Sold"],df_silver["RevenuePerUnit"],df_branch["dim_branch_key"],df_dealer["dim_Dealer_key"],df_model["dim_surrogate_key"],df_date["dim_Date_key"])

In [0]:
df_fact.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists('fact_sales'):
    deltatbl = Deltatable.forname(spark,'cars_catalog.gold_schema.fact_sales')

    deltatbl.alias("trg").merge(df_fact.alias("src"), "trg.dim_branch_key = src.dim_branch_key and trg.dim_Dealer_key = src.dim_Dealer_key and trg.dim_surrogate_key = src.dim_surrogate_key and trg.dim_Date_key = src.dim_Date_key")\
    .whenMatchedUpdateAll()\
    .whenNotMatchedInsertAll()

else:
    df_fact.write.format("delta")\
        .mode("Overwrite")\
        .option('path',"abfss://gold@datastorager.dfs.core.windows.net/fact_sales")\
        .saveAsTable("cars_catalog.gold_schema.fact_sales")

In [0]:
%sql
select * from cars_catalog.gold_schema.fact_sales;